# City street network orientations

Author: [Geoff Boeing](https://geoffboeing.com/)

Compare the spatial orientations of city street networks with OSMnx.

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd

ox.config(log_console=True, use_cache=True)
weight_by_length = False

ox.__version__

In [ ]:
# define the study sites as label : query
places = {'Barking & Dagenham':'London Borough of Barking and Dagenham',
          'Barnet':'London Borough of Barnet',
          'Bexley':'London Borough of Bexley',
          'Brent':'London Borough of Brent',
          'Bromley':'London Borough of Bromley',
          'Camden':'London Borough of Camden',
          'Croydon':'London Borough of Croydon',
          'Ealing':'London Borough of Ealing',
          'Enfield':'London Borough of Enfield',
          'Greenwich':'Royal Borough of Greenwich',
          'Hackney':'London Borough of Hackney',
          'Hammersmith & Fulham':'London Borough of Hammersmith and Fulham',
          'Haringey':'London Borough of Haringey',
          'Harrow':'London Borough of Harrow',
          'Havering':'London Borough of Havering',
          'Hillingdon':'London Borough of Hillingdon',
          'Hounslow':'London Borough of Hounslow',
          'Islington':'London Borough of Islington',
          'Kensington & Chelsea':'Royal Borough of Kensington and Chelsea',
          'Kingston upon Thames':'Royal Borough of Kingston upon Thames',
          'Lambeth':'London Borough of Lambeth',
          'Lewisham':'London Borough of Lewisham',
          'Merton':'London Borough of Merton',
          'Newham':'London Borough of Newham',
          'Redbridge':'London Borough of Redbridge',
          'Richmond upon Thames':'London Borough of Richmond upon Thames',
          'Southwark':'London Borough of Southwark',
          'Sutton':'London Borough of Sutton',
          'Tower Hamlets':'London Borough of Tower Hamlets',
          'Waltham Forest':'London Borough of Waltham Forest',
          'Wandsworth':'London Borough of Wandsworth',
          'City of Westminster':'City of Westminster'}
         

In [ ]:
# verify OSMnx geocodes each query to what you expect (a [multi]polygon geometry)
gdf = ox.gdf_from_places(places.values())
gdf

## Get the street networks and their edge bearings

In [ ]:
def reverse_bearing(x):
    return x + 180 if x < 180 else x - 180

In [ ]:
bearings = {}
for place in sorted(places.keys()):
    print(datetime.datetime.now(), place)
    
    # get the graph
    query = places[place]
    G = ox.graph_from_place(query, network_type='drive')
    
    # calculate edge bearings
    Gu = ox.add_edge_bearings(ox.get_undirected(G))
    
    if weight_by_length:
        # weight bearings by length (meters)
        city_bearings = []
        for u, v, k, d in Gu.edges(keys=True, data=True):
            city_bearings.extend([d['bearing']] * int(d['length']))
        b = pd.Series(city_bearings)
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')
    else:
        # don't weight bearings, just take one value per street segment
        b = pd.Series([d['bearing'] for u, v, k, d in Gu.edges(keys=True, data=True)])
        bearings[place] = pd.concat([b, b.map(reverse_bearing)]).reset_index(drop='True')

## Visualize it

In [ ]:
def count_and_merge(n, bearings):
    # make twice as many bins as desired, then merge them in pairs
    # prevents bin-edge effects around common values like 0° and 90°
    n = n * 2
    bins = np.arange(n + 1) * 360 / n
    count, _ = np.histogram(bearings, bins=bins)
    
    # move the last bin to the front, so eg 0.01° and 359.99° will be binned together
    count = np.roll(count, 1)
    return count[::2] + count[1::2]

In [ ]:
# function to draw a polar histogram for a set of edge bearings
def polar_plot(ax, bearings, n=36, title=''):

    bins = np.arange(n + 1) * 360 / n
    count = count_and_merge(n, bearings)
    _, division = np.histogram(bearings, bins=bins)
    frequency = count / count.sum()
    division = division[0:-1]
    width =  2 * np.pi / n

    ax.set_theta_zero_location('N')
    ax.set_theta_direction('clockwise')

    x = division * np.pi / 180
    bars = ax.bar(x, height=frequency, width=width, align='center', bottom=0, zorder=2,
                  color='#df2a2a', edgecolor='k', linewidth=0.5, alpha=0.7)
    
    ax.set_ylim(top=frequency.max())
    
    title_font = {'family':'P22 Johnston Underground', 'size':22, 'weight':'bold'}
    xtick_font = {'family':'P22 Johnston Underground', 'size':10, 'weight':'bold', 'alpha':1.0, 'zorder':3}
    ytick_font = {'family':'P22 Johnston Underground', 'size': 9, 'weight':'bold', 'alpha':0.2, 'zorder':3}
    
    ax.set_title(title.upper(), y=1.05, fontdict=title_font)
    
    ax.set_yticks(np.linspace(0, max(ax.get_ylim()), 5))
    yticklabels = ['{:.2f}'.format(y) for y in ax.get_yticks()]
    yticklabels[0] = ''
    ax.set_yticklabels(labels=yticklabels, fontdict=ytick_font)
    
    xticklabels = ['N', '', 'E', '', 'S', '', 'W', '']
    ax.set_xticklabels(labels=xticklabels, fontdict=xtick_font)
    ax.tick_params(axis='x', which='major', pad=-2)

In [ ]:
# create figure and axes
n = len(places)
ncols = int(np.ceil(np.sqrt(n)))
nrows = int(np.ceil(n / ncols))
figsize = (ncols * 5, nrows * 5)
fig, axes = plt.subplots(nrows, ncols, figsize=figsize, subplot_kw={'projection':'polar'})

# plot each city's polar histogram
for ax, place in zip(axes.flat, sorted(places.keys())):
    polar_plot(ax, bearings[place].dropna(), title=place)

# add super title and save full image
suptitle_font = {'family':'P22 Johnston Underground', 'fontsize':50, 'fontweight':'normal', 'y':1.07}
fig.suptitle('Street Network Orientation: London Boroughs', **suptitle_font)
fig.tight_layout()
fig.subplots_adjust(hspace=0.35)
fig.savefig('images/london_street_orientations.png', dpi=500, bbox_inches='tight')
plt.close()